In [27]:
import tensorflow as tf

In [28]:
input_x = tf.placeholder(tf.float32, [64,192, 3+1], name = 'input_x')
cnn_input = input_x[:, :168, 0] #shape = [batch_size, encode_length], pure data
cnn_input = tf.expand_dims(cnn_input, axis=2)
cnn_input = tf.expand_dims(cnn_input, axis=3) #[batch_size, encode_length, width=1, input_channel=1]
print ("cnn_input.shape: ", cnn_input.shape)

cnn_input.shape:  (64, 168, 1, 1)


In [30]:

### Conv ###

filter_size_list = [3, 3, 3, 3, 3] # 看前后
output_channel_list = [80,60,40,40,40]
input_channel_list = [1,80,60,40,40]
cnn_output_list = []
cnn_input_list = []
cnn_input_list.append(cnn_input)
print((cnn_input_list[0].shape))
batch_size = 64
dialations_size = [1, 3, 6, 12, 24]
padding_size = [2, 6, 12, 24, 48]

(64, 168, 1, 1)


In [31]:
for i, filter_size in enumerate(filter_size_list):
    with tf.variable_scope("CNN-"+str(filter_size)):
        print()
        num_padding = padding_size[i]
        pad = tf.zeros([batch_size, num_padding, 1, input_channel_list[i]], tf.float32)
        print("pad.shape: ",pad.shape)
        print("intput.shape: ",cnn_input_list[i].shape)
        batch_input_conv_pad = tf.concat([pad, cnn_input_list[i]], 1) #[batch_size, encode_length+filter_size-1, width=1, input_channel=1]
        print ("batch_input_conv_pad.shape: ", batch_input_conv_pad.shape)
        #print ("cnn_input.shape: ", cnn_input.shape)
        filter_shape = [filter_size, 1, input_channel_list[i], output_channel_list[i]]
        # parameter for conv
        W = tf.Variable(
            tf.truncated_normal(filter_shape, stddev=0.1), name='W')
        b = tf.Variable(
            tf.constant(0.1, shape = [output_channel_list[i]]), name='b')
        conv = tf.nn.conv2d(
            batch_input_conv_pad,  ### shape = [batch_size, encode_length]
            W,
            strides=[1,1,1,1],
            padding = 'VALID', #不进行padding，多余部分丢弃
            name='conv',
            dilations = [1,dialations_size[i],1,1]
            #dilations = [1,1,1,1]
            ) #conv.shape: [batch_size, sequence_length(因为stride是1),1, num_filters(也就是feature maps的个数)]
        h = tf.nn.relu(tf.nn.bias_add(conv,b), name = 'relu') # [batch_size, encode_length, width=1, ouput_channel = 40]
        cnn_input_list.append(h)
        h = tf.squeeze(h, [2]) # [batch_size, encode_length, ouput_channel = 40]
        cnn_output_list.append(h)
        print ("h.shape: ", h.shape)
#cnn_output = tf.concat(cnn_output_list, axis=2) #[batch_size, encode_length, num_filter*4=40]
cnn_output = cnn_output_list[-1]
print ("===============\ncnn_output.shape: ", cnn_output.shape)


pad.shape:  (64, 2, 1, 1)
intput.shape:  (64, 168, 1, 1)
batch_input_conv_pad.shape:  (64, 170, 1, 1)
h.shape:  (64, 168, 80)

pad.shape:  (64, 6, 1, 80)
intput.shape:  (64, 168, 1, 80)
batch_input_conv_pad.shape:  (64, 174, 1, 80)
h.shape:  (64, 168, 60)

pad.shape:  (64, 12, 1, 60)
intput.shape:  (64, 168, 1, 60)
batch_input_conv_pad.shape:  (64, 180, 1, 60)
h.shape:  (64, 168, 40)

pad.shape:  (64, 24, 1, 40)
intput.shape:  (64, 168, 1, 40)
batch_input_conv_pad.shape:  (64, 192, 1, 40)
h.shape:  (64, 168, 40)

pad.shape:  (64, 48, 1, 40)
intput.shape:  (64, 168, 1, 40)
batch_input_conv_pad.shape:  (64, 216, 1, 40)
h.shape:  (64, 168, 40)
cnn_output.shape:  (64, 168, 40)
